# Labeling with GPT4o

This notebook labels the samples using the GPT4o model. 

In [5]:
import os 
from openai import OpenAI

KEY = os.environ["OPENAI_KEY"] 

Given the length of the samples could affect the labeling performance, I computed the token length of the samples using [Open AI online tokenizer](https://platform.openai.com/tokenizer) and conducted a literature review to find the optimal length of the samples for the GPT4o model, the related papers are listed below. It seems to me our prompt and samples are short enough to be processed in batches. 

BABILong: a long-context needle-in-a-haystack benchmark for LLMs

https://github.com/booydar/babilong

https://arxiv.org/pdf/2404.02060

https://github.com/TIGER-AI-Lab/LongICLBench



In [35]:
system_message = """You are a crowd-source worker who identifies and label hate speech in Turkish tweets. For each tweet, find the hate speech spans targeting the given group and label them with the hate speech category.
Hate Speech Categories:
- Exclusive/Discriminatory Discourse
- Exaggeration; Generalization; Attribution; Distortion 
- Threat of Enmity; War; Attack; Murder; or Harm 
- Symbolization 
- Swearing; Insult; Defamation; Dehumanization
Return a JSON for each tweet:
{
  "hate_speech_instances": [
    { 
      "tweet": {
         "id": "",
		 "span": [ 
            "text_span": "",
            "hate_speech_category": ""
		] 
	  }
    },
	{
	   "tweet": {
         "id": "",
		 "span": [ 
            "text_span": "",
            "hate_speech_category": ""
		] 
	  }
	}
  ]
}
"""

def query_gpt(text):

	
	client = OpenAI(
	    # This is the default and can be omitted
	    api_key=KEY,
	)

	response = client.chat.completions.create(
	    messages=[
	    	{
	    		"role": "system",
	    		"content": system_message,
			},
	        {
	            "role": "user",
	            "content": text,
	        }
	    ],
	    model="gpt-4o-2024-05-13", #"gpt-4o-2024-05-13",
	)

	return response.choices[0].message.content


# tweet = '''"
# Tweet id: 1, Target: Kurdish, Text: @shemhamfhorash @aIexstrasza_ @xdomerxd @pusholder Gönül isterdi ki Sinan Oğan kazansın ya da c*p hdpkk safında durmasın, %10 kürt oyu toplayayım diye darmaduman oldular. Beter olsunlar, bunların karşısında bırak RTE'yi taş olsa taşa basarım.\nhttps://t.co/3hHhDqODrr
# Tweet id: 2, Target: Refugee, Text:  Yurt önünde türk yok türk... içeriye anonsla alan bile suriyeli. Bu ne biçim iş. #ülkemdemülteciistemiyorum
# Tweet id: 3, Target: Greek, Text: @username Yunanistan'ın ne kadar borcu var? 1 trilyon dolar? 😂 Umarım savaş başlar ve Türk ordusunun acı gücünü dedelerinizin izmirde tattığı gibi sizde tadarsınız!!! Bu kadar klavye konuşması yeterli. Umarım savaş meydanında karşılaşırız yorgo 😂
# Tweet id: 4, Target: Jew, Text: Filistin-Mısır sınırına itrail saldırı düzenlendi orada da müslümanlar sınıra dayandı diye onlara da saldırmaya başladı kuduz köpek.\n#palestine_egypt\n#filistin_misir https://t.co/r46clGBMs9
# Tweet id: 5, Target: Arab, Text:  @FbVit44 @emredem03 @LuckyBesta Arap sevici yallah çölüne burası Türkiye ve ne mutlu türküm diyeceğiz yallah sen
# Tweet id: 6, Target: Jew, Text: Siz İsrail oğulları hiç mi geçmişinizden ders almazsınız ebabil kuşları nasıl bir fil ordusun yok ettiyse sizinde sonunuzda onların ki gibi olacak !\n\n    #AksadaBaskinVar
# Tweet id: 7, Target: Greek, Text: Yunanistan provokasyon Rumlar kışkırtma peşinde https://t.co/qtUy24gmxq @username aracılığıyla
# Tweet id: 8, Target: Jew, Text: Yorumlarda hintliler " İsrail askeri olarak savşmak istiyorum" yazmışlar. Nedir bu hintlilerdeki itrail aşkı? https://t.co/aO1F2ks4lv
# Tweet id: 9, Target: Refugee, Text: Ya 2-3 gündür denk geliyorum şu suriyeli mevzusunu savunanların çoğunun kadın olması... Yapmayın ablalar yapmayın. Çıkın o elit semtlerinizden suriyeliler afganlar bildiğiniz gibi değil https://t.co/SNjWunjZCe	
# Tweet id: 10, Target: Greek, Text: @Ahaber Dünya Tarihinin En Azametli Tarihini Yazar Şanlı Ecdadimizin Adını Destursuz anan serseriler\n\n""Bu milletin Ferdi Değillerdir""\n\nYoksa kimse Kendi Arasına Sövecek Kadar Aşağılıklaşamaz \n\nBunlar Olsa olsa Yunan Dölü Olması Lazm getir... 
# '''
# output = query_gpt(tweet)

In [36]:
import pandas as pd

samples = pd.read_csv('../data/span_samples_v3.csv')
samples

,genel_tutum,hedef_grup,derece,kategori,saldırgan_dil,tweet_text,annotator,uploaded_at,konu,tweet_id,dil,kategori_count
0,Anti-Immigrant/Refugee,['Country/Nationality'],6,['Swearing; Insult; Defamation; Dehumanization...,Low,بالآونه الاخيرة اكثر الحسابات المتنكرة كسعوديي...,Neda,2024-02-24T15:56:32,Arabic,1659141226585571331,NaN,2
1,Anti-Immigrant/Refugee,['Country/Nationality'],7,['Exaggeration; Generalization; Attribution; D...,Low,دخولهم الى دولنا خطير بسبب \nاعمال السحر والشع...,Neda,2023-11-14T07:57:28,Arabic,1659158751507521537,NaN,2
2,Arap Karşıtı,"['Irk/Etnik Köken', 'Ülke/Milliyet']",5,['Abartma; Genelleme; Yükleme; Çarpıtma'],Yok,@muhendisyenn Hemde çok büyük bir sorun var. B...,Şura,2023-11-14T07:57:40,Arabian,1658061958652755968,Türkçe,1
3,Arap Karşıtı,['Irk/Etnik Köken'],4,['Simgeleştirme'],Zayıf,En milliyetçi adamlardan biriyim ama asla mill...,Firdevs,2023-08-25T19:01:15,Arabian,1658543138355461888,Türkçe,1
4,Arap Karşıtı,['Irk/Etnik Köken'],3,['Abartma; Genelleme; Yükleme; Çarpıtma'],Yok,@championoneday @secimtr2023 Kendi milletimden...,Şura,2023-08-17T21:14:31,Arabian,1658150712864387072,Türkçe,1
...,...,...,...,...,...,...,...,...,...,...,...,...
247,Yunan Karşıtı,['Irk/Etnik Köken'],7,['Simgeleştirme'],Şiddetli,@ARZU61__ @fikretk53429081 @YukselisGruplar #Y...,Ebrar,2023-02-01T17:44:12,Tr-Gr,1568670348182265857,Türkçe,1
248,Yunan Karşıtı,['Irk/Etnik Köken'],8,"['Simgeleştirme', 'Düşmanlık; Savaş; Saldırı; ...",Şiddetli,"HEPSİ YUNAN TOHUMU ,HEPSİ TERÖRİST https://t.c...",Aygün,2023-01-10T20:01:33,Tr-Gr,1530572228701704193,NaN,2
249,Yunan Karşıtı,"['Irk/Etnik Köken', 'Ülke/Milliyet', 'Din', 'H...",7,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Şiddetli,@username SOYSUZ SOYKIRIMCI YUNAN.MÜSLÜMAN TÜR...,Altar,2023-07-22T20:25:42,Tr-Gr,1559088833341411330,Türkçe,1
250,Yunan Karşıtı,['Irk/Etnik Köken'],6,['Küfür; Hakaret; Aşağılama; İnsandışılaştırma'],Şiddetli,@username @username @username Sen anca bok yer...,İpek,2023-03-22T10:38:48,Tr-Gr,1522835116254547969,Türkçe,1


In [37]:
translations = {
    'Göçmen/Mülteci Karşıtı': 'Immigrant/Refugee',
    'Yunan Karşıtı': 'Greek',
    'Yahudi Karşıtı': 'Jew',
    'Ermeni Karşıtı': 'Armenian',
    'Arap Karşıtı': 'Arab',
    'LGBTI+ Karşıtı': 'LGBTI+',
    'Kürt Karşıtı': 'Kurdish',
    'Alevi Karşıtı': 'Alevi'
}

samples['target'] = samples['genel_tutum'].map(translations)

In [39]:
targets = samples['target'].unique()
targets

array([nan, 'Arab', 'Armenian', 'Immigrant/Refugee', 'Jew', 'Greek'],
      dtype=object)

In [40]:
samples['model_input'] = samples.apply(lambda x: f"Tweet id: {x['tweet_id']}, Target: {x['target']}, Text: {x['tweet_text']}", axis=1)

In [41]:
samples = samples.dropna(subset=['target'])
samples['target'].value_counts()

Arab                 50
Armenian             50
Immigrant/Refugee    50
Jew                  50
Greek                50
Name: target, dtype: int64

In [42]:
print(samples[samples['target'] == 'Jew']['model_input'].str.cat(sep='\n'))

Tweet id: 1555925857637699585, Target: Jew, Text: Ey İnsanlık Nerdesin❓
Ey Dünya Niye Sessizsin❓

İsrail ordusunun, #Gazze’ye yönelik saldırıları devam ediyor❗

Saldırılarda aralarında 5 yaşında bir çocuğun da buluduğu 12 masum hayatını kaybetti, 84 kardeşimiz de yaralandı.

İsrail'i Şiddetle Kınıyorum.

#GazzeAtesAltında https://t.co/FJAlBLNuJZ
Tweet id: 1556025770488774658, Target: Jew, Text: Çocuk katilleri!
Lanetli kavim!
Çöllere düşün son kez ve bir daha çıkamayın.

#GazaUnderAttack 
#GazzeAtesAltında 
#Gazze
Tweet id: 1473971550542090246, Target: Jew, Text: Bu itrail her seyiyle insanlığın yüzkarasi. #RobertMaloneYalnızDeğilsin https://t.co/6NY2XiLpQ6
Tweet id: 1393308652879196160, Target: Jew, Text: 15 SANİYEDE NELER OLUR😌

İşgalci #Teröristİsrail’in #Gazze Şeridi‘ne yönelik yoğun bombardıman devam ediyor 

Uyanma vakti!
Filistin yalnız değildir.

#WakeUp
#GazzaUnderAttack 
#GenocideinGaza
@username @username 
#FilistinYalnızDeğil #MilletinAdamı https://t.co/Rcj05Yhu6t
Tweet id:

In [43]:
for target in samples['target'].unique(): # ['Alevi', 'Kurdish', 'Armenian', 'LGBTI+', 'Arab', 'Greek', 'Jew']
    text = samples[samples['target'] == target]['model_input'].str.cat(sep='\n')
    output = query_gpt(text)
    with open(f'../data/gpt4o_outputs/{target.replace("/", "-")}_v3.json', 'w', encoding='utf-8') as f:
        f.write(output)

In [44]:
# dictionary to tweet id to text
id2text = samples.set_index('tweet_id')['tweet_text'].to_dict()

In [46]:
from pathlib import Path    
import json 
for target in samples['target'].unique():
    print(target)
    with open(f'../data/gpt4o_outputs/{target.replace("/", "-")}_v3.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
        for instance in data['hate_speech_instances']:
            tweet_id = instance['tweet']['id']
            try: 
                instance['tweet']['text'] = id2text[int(tweet_id)]
            except KeyError:
                print(f"Tweet id {tweet_id} not found")
    with open(f'../data/span_annotations/{target.replace("/", "-")}_v3.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

Arab
Armenian
Immigrant/Refugee
Jew
Greek


In [47]:
import json 
import pandas as pd 


for target in samples['target'].unique():
    print(target)
    records = []
    with open(f'../data/span_annotations/{target.replace("/", "-")}_v3.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
        instances = data['hate_speech_instances']
        for instance in instances:
            try:
                tweet_id = instance['tweet']['id']
                text = instance['tweet']['text']
                spans = instance['tweet']['span']
                span_txt = ",".join([f"{span['text_span']}" for span in spans])
                category_txt = ",".join([f"{span['hate_speech_category']}" for span in spans])
                records.append([tweet_id, target, text, span_txt, category_txt])
            except KeyError:
                print(f"Tweet id {tweet_id} not found")
    df = pd.DataFrame(records, columns=['tweet_id', 'target', 'text', 'span', 'category'])
    df.to_csv(f'../data/span_annotations/{target.replace("/", "-")}_v3.csv', index=False)

Arab
Armenian
Immigrant/Refugee
Jew
Greek
